In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = "https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv"
headers = {"Accept-Language": "en-US, en;q=0.5"}
results = requests.get(url, headers=headers)

soup = BeautifulSoup(results.text, "html.parser")

#print(soup)

titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

movie_div = soup.find_all('div', class_='lister-item mode-advanced')

for container in movie_div:

        #name
        name = container.h3.a.text
        titles.append(name)
        
        #year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)

        #runtime
        runtime = container.p.find('span', class_='runtime').text if container.p.find('span', class_='runtime').text else '-'
        time.append(runtime)

        #IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        #metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)

        #There are two NV containers, grab both of them as they hold both the votes and the grosses
        nv = container.find_all('span', attrs={'name': 'nv'})
        
        #filter nv for votes
        vote = nv[0].text
        votes.append(vote)
        
        #filter nv for gross
        grosses = nv[1].text if len(nv) > 1 else '-'
        
        us_gross.append(grosses)

#building our Pandas dataframe         
movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': time,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
})

#cleaning data with Pandas
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)
movies['metascore'] = movies['metascore'].astype(int)
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)
movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

# if you want to save pd.concat([movies['year'], movies['metascore'], movies['us_grossMillions']]).to_csv('foo.csv')

print(movies)
movies.shape
movies.ndim

movies.head()


                                                movie  year  timeMin  imdb  \
0                                            Parasite  2019      132   8.6   
1                                          Knives Out  2019      131   8.0   
2                                         Jojo Rabbit  2019      108   8.0   
3                                                1917  2019      119   8.4   
4                                       The Gentlemen  2019      113   8.1   
5                                          Uncut Gems  2019      135   7.6   
6                    Once Upon a Time... in Hollywood  2019      161   7.7   
7                                               Joker  2019      122   8.6   
8                                      Ford v Ferrari  2019      152   8.1   
9                                        Little Women  2019      135   8.0   
10                                     The Lighthouse  2019      109   7.7   
11                                       The Irishman  2019     

,movie,year,timeMin,imdb,metascore,votes,us_grossMillions
0,Parasite,2019,132,8.6,96,294274,NaN
1,Knives Out,2019,131,8.0,82,205459,NaN
2,Jojo Rabbit,2019,108,8.0,58,149640,0.35
3,1917,2019,119,8.4,78,205939,NaN
4,The Gentlemen,2019,113,8.1,51,46321,NaN
